In [1]:
def ClusterRun(function, parameter_list, max_cores=100):
    '''function: The routine run in parallel, which must contain all necessary
       imports internally.
    
       parameter_list: should be an iterable of elements, for which each element
       will be passed as the parameter to function for each parallel execution.
       
       max_cores: Standard Rhino cluster etiquette is to stay within 100 cores
       at a time.  Please ask for permission before using more.
       
       In jupyterlab, the number of engines reported as initially running may
       be smaller than the number actually running.  Check usage from an ssh
       terminal using:  qstat -f | egrep "$USER|node" | less
       
       Undesired running jobs can be killed by reading the JOBID at the left
       of that qstat command, then doing:  qdel JOBID
    '''
    import cluster_helper.cluster
    from pathlib import Path

    num_cores = len(parameter_list)
    num_cores = min(num_cores, max_cores)

    myhomedir = str(Path.home())

    with cluster_helper.cluster.cluster_view(scheduler="sge", queue="RAM.q", \
        num_jobs=1, cores_per_job=num_cores, \
        extra_params={'resources':'pename=python-round-robin'}, \
        profile=myhomedir + '/.ipython/') \
        as view:
        # 'map' applies a function to each value within an interable
        res = view.map(function, parameter_list)
        
    return res


In [2]:
def squared(x):
    try:
        import numpy as np
    
        res = x**2
        
        # You can directly return results, but saving them in separate files is
        # better practice for large jobs.
        np.save('squared_result_'+str(x)+'.npy', res)
        
        # We return True for success
        return True
    
    except Exception as e:
        # Handle your exceptions appropriately.
        # For large runs, you will probably want to write the exception to a log file.
        # Try to handle more exceptions closer to where they happen, so that few
        # get here to the top level.
        
        np.savetxt('squared_error_'+str(x)+'.txt', \
                   [str(x), str(e.__class__.__name__), str(e)], \
                   fmt='%s')
        return False
    

parameters = range(0, 10)
ClusterRun(squared, parameters)

1 Engines running
Sending a shutdown signal to the controller and engines.


[True, True, True, True, True, True, True, True, True, True]

In [3]:
import numpy as np
for x in parameters:
    print(np.load('squared_result_'+str(x)+'.npy', allow_pickle=True))

0
1
4
9
16
25
36
49
64
81
